In [1]:
import os
import time
from tqdm import *

import torch
import torchvision.utils as vutils
from torch.autograd import Variable
import torchvision
import numpy as np
from utilFuncs import one_hot_encode

import torch.utils.data as Data

from CapsNet import CapsNet

In [2]:
BATCH_SIZE = 64
EPOCH = 50

In [3]:
dataset = torchvision.datasets.CIFAR10(
    root = './cifar10',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True
)

Files already downloaded and verified


In [4]:
train_loader = Data.DataLoader(dataset=dataset,batch_size = BATCH_SIZE, shuffle = True)

In [5]:
testset = torchvision.datasets.CIFAR10(
    root = './cifar10',
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = False
)

In [6]:
test_loader = Data.DataLoader(dataset=testset,batch_size = BATCH_SIZE, shuffle = True)

In [7]:
net = CapsNet(True).cuda()

In [8]:
optimizer = torch.optim.Adam(net.parameters(),lr=0.03)

In [9]:
for epoch in range(EPOCH):
    avg_cost = 0
    margin_L = 0
    recons_L = 0
    for step,(x,y) in enumerate(train_loader):
        batch_size = y.size(0)
        y = one_hot_encode(y,10)
        x,y = Variable(x).cuda(),Variable(y).cuda()
        optimizer.zero_grad()
        output = net(x)
        
        Loss,m_loss,r_loss = net.loss(output,y,x)
        Loss.backward()
        
        optimizer.step()
        
        avg_cost += Loss
        margin_L += m_loss
        recons_L += r_loss
    if epoch%5 == 0:
        avg_cost = avg_cost/(step+1)*x.size(0)
        margin_L = margin_L/(step+1)*x.size(0)
        recons_L = recons_L/(step+1)*x.size(0)
        print 'Epoch:',epoch,',loss:',avg_cost.data,'margin loss:',margin_L,'reconstruction loss:',recons_L
        
    if epoch>30 and epoch%10==0:
        model_out_path = './cifar10_epoch_{}.pth'.format(epoch)
        torch.save({
            'epoch': epoch + 1,
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict()
            }, model_out_path)
        

/home/hewenrui/.conda/envs/hewenrui/lib/python2.7/site-packages/torch/autograd/_functions/basic_ops.py:48: UserWarning: self and other not broadcastable, but have the same number of elements.  Falling back to deprecated pointwise behavior.
  return a.mul(b)


Epoch: 0 ,loss: 
 33.7441
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 margin loss: Variable containing:
 33.3033
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 reconstruction loss: Variable containing:
 0.4408
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Epoch: 5 ,loss: 
 27.8585
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 margin loss: Variable containing:
 27.4465
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 reconstruction loss: Variable containing:
 0.4120
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Epoch: 10 ,loss: 
 27.5998
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 margin loss: Variable containing:
 27.1981
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 reconstruction loss: Variable containing:
 0.4016
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Epoch: 15 ,loss: 
 27.4701
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 margin loss: Variable containing:
 27.0723
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 reconstruction loss: Variable containing:
 0.3978
[torch.cuda.FloatTensor of size 1 (GPU

In [14]:
correct = 0.
for step,(x_t,y_t) in enumerate(test_loader):
    y_t_var = Variable(one_hot_encode(y_t,10)).cuda()
    x_t_var = Variable(x_t).cuda()
    output = net(x_t_var)
    batch_size = output.size(0)
    predict = torch.sqrt(torch.sum(output**2,dim=-2)).squeeze()
    _,predict = torch.max(predict,dim=1)
    batch_correct=predict.data.eq(y_t.type(torch.cuda.LongTensor).view_as(predict.data)).sum()
    correct += batch_correct
    print 'epoch acc:',batch_correct/batch_size
acc = correct/len(test_loader.dataset)
print acc

epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0
epoch acc: 0


KeyboardInterrupt: 